# Kaggle - Porte Seguro Safe Driver Prediction - Perfomance Improvement 2
**Author: Chris Shin**

In this improvement, we will use XGB to compare the performance.


Advantages of LightGBM:

- Faster training speed and lower memory usage compared to other algorithms, as it uses a novel technique called Gradient-based One-Side Sampling (GOSS) for handling imbalanced data.
- Highly scalable and efficient for large datasets with high-dimensional features.
- Supports parallel and distributed computing.
- Can handle categorical features natively without the need for one-hot encoding.

Disadvantages of LightGBM:

- Prone to overfitting if the dataset is not properly tuned or regularized.
- Lack of interpretability as it uses a tree-based model.

Advantages of XGBoost:

- Powerful and widely used algorithm with high accuracy on many datasets.
- Robust to noise and outliers in the data.
- Supports parallel and distributed computing.
- Provides built-in regularization to prevent overfitting.
- Offers better interpretability than neural networks.

Disadvantages of XGBoost:

- Slower than LightGBM in training time and memory usage for large datasets.
- Limited support for handling categorical features.


Advantages of Random Forest:

- Simple and easy to implement.
- Less prone to overfitting compared to decision trees.
- Can handle high-dimensional datasets and noisy data.
- Provides feature importance ranking.


Disadvantages of Random Forest:

- Can be slow for training and prediction compared to other algorithms.
- Limited ability to learn complex patterns in the data compared to neural networks or gradient boosting algorithms.
- Not suitable for datasets with high-cardinality categorical features.


When to use each algorithm:

- LightGBM or XGBoost are good choices for large datasets with high-dimensional features and imbalanced classes. Use - - LightGBM if speed and memory efficiency are a concern, and use XGBoost if interpretability and handling sparse data are more important.
- Random Forest is a good choice for small to medium-sized datasets with moderate complexity and non-linear relationships between the features and target variable. It can also be used for feature selection and importance ranking.

In [1]:
import pandas as pd
pd.options.display.max_columns = None

train = pd.read_csv('./data/train.csv', index_col='id')
test = pd.read_csv('./data/test.csv', index_col='id')
submission = pd.read_csv('./data/sample_submission.csv', index_col='id')

In [2]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1)

all_features = all_data.columns

In [3]:
from sklearn.preprocessing import OneHotEncoder

cat_features = [feature for feature in all_features if 'cat' in feature] 

onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features]) 

In [4]:
all_data['num_missing'] = (all_data==-1).sum(axis=1)

In [5]:
remaining_features = [feature for feature in all_features
                      if ('cat' not in feature and 'calc' not in feature)] 
remaining_features.append('num_missing')

In [6]:
ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True
for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str) + '_'
        is_first_feature = False
    else:
        all_data['mix_ind'] += all_data[ind_feature].astype(str) + '_'

In [7]:
cat_count_features = []
for feature in cat_features+['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x: 
                                                           val_counts_dict[x])
    cat_count_features.append(f'{feature}_count')

In [8]:
from scipy import sparse
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 
                 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                              format='csr')

In [9]:
num_train = len(train)

X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

In [10]:
import numpy as np

def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0]
    L_mid = np.linspace(1 / n_samples, 1, n_samples)

    pred_order = y_true[y_pred.argsort()]
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) 
    G_pred = np.sum(L_mid - L_pred) 

    true_order = y_true[y_true.argsort()]
    L_true = np.cumsum(true_order) / np.sum(true_order) 
    G_true = np.sum(L_mid - L_true)

    return G_pred / G_true

In [11]:
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds)

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      random_state=0)
bayes_dtrain = xgb.DMatrix(X_train, y_train)
bayes_dvalid = xgb.DMatrix(X_valid, y_valid)

In [13]:
param_bounds = {'max_depth': (4, 8),
                'subsample': (0.6, 0.9),
                'colsample_bytree': (0.7, 1.0),
                'min_child_weight': (5, 7),
                'gamma': (8, 11),
                'reg_alpha': (7, 9),
                'reg_lambda': (1.1, 1.5),
                'scale_pos_weight': (1.4, 1.6)}

fixed_params = {'objective': 'binary:logistic',
                'learning_rate': 0.02,
                'random_state': 1991}

In [14]:
def eval_function(max_depth, subsample, colsample_bytree, min_child_weight,
                 reg_alpha, gamma, reg_lambda, scale_pos_weight):

    params = {'max_depth': int(round(max_depth)),
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'min_child_weight': min_child_weight,
              'gamma': gamma,
              'reg_alpha':reg_alpha,
              'reg_lambda': reg_lambda,
              'scale_pos_weight': scale_pos_weight}

    params.update(fixed_params)
    
    print('hyperparameters :', params)    
        
    xgb_model = xgb.train(params=params, 
                          dtrain=bayes_dtrain,
                          num_boost_round=2000,
                          evals=[(bayes_dvalid, 'bayes_dvalid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=False)
                           
    best_iter = xgb_model.best_iteration
    preds = xgb_model.predict(bayes_dvalid, 
                              iteration_range=(0, best_iter))

    gini_score = eval_gini(y_valid, preds)
    print(f'Gini Coefficient : {gini_score}\n')
    
    return gini_score

In [15]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(f=eval_function, 
                                 pbounds=param_bounds, 
                                 random_state=0)

optimizer.maximize(init_points=3, n_iter=6)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------
hyperparameters : {'max_depth': 6, 'subsample': 0.867531900234624, 'colsample_bytree': 0.8646440511781974, 'min_child_weight': 6.0897663659937935, 'gamma': 10.14556809911726, 'reg_alpha': 7.84730959867781, 'reg_lambda': 1.3583576452266626, 'scale_pos_weight': 1.4875174422525386, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


c:\Users\Yuchie\Desktop\DataScience\kaggle_env\Lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


Gini Coefficient : 0.2852804659784522

| 1         | 0.2853    | 0.8646    | 10.15     | 6.411     | 6.09      | 7.847     | 1.358     | 1.488     | 0.8675    |
hyperparameters : {'max_depth': 7, 'subsample': 0.6261387899104622, 'colsample_bytree': 0.9890988281503088, 'min_child_weight': 6.0577898395058085, 'gamma': 9.150324556477333, 'reg_alpha': 8.136089122187865, 'reg_lambda': 1.4702386553170643, 'scale_pos_weight': 1.4142072116395774, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}
Gini Coefficient : 0.2848088779310762

| 2         | 0.2848    | 0.9891    | 9.15      | 7.167     | 6.058     | 8.136     | 1.47      | 1.414     | 0.6261    |
hyperparameters : {'max_depth': 7, 'subsample': 0.8341587528859367, 'colsample_bytree': 0.7060655192320977, 'min_child_weight': 6.7400242964936385, 'gamma': 10.497859536643814, 'reg_alpha': 8.957236684465528, 'reg_lambda': 1.4196634256866894, 'scale_pos_weight': 1.4922958724505864, 'objective': 'binary:logistic', 'lea

c:\Users\Yuchie\Desktop\DataScience\kaggle_env\Lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


Gini Coefficient : 0.28372532851421195

| 4         | 0.2837    | 0.8843    | 10.45     | 6.838     | 6.494     | 8.552     | 1.381     | 1.423     | 0.7002    |
hyperparameters : {'max_depth': 7, 'subsample': 0.8535233675350644, 'colsample_bytree': 0.92975858050776, 'min_child_weight': 6.249564429359247, 'gamma': 9.95563546750357, 'reg_alpha': 8.411512219837842, 'reg_lambda': 1.424460008293778, 'scale_pos_weight': 1.5416807226581535, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


...

Gini Coefficient : 0.28467572646560496

| 7         | 0.2847    | 0.8818    | 9.013     | 6.927     | 6.992     | 7.641     | 1.356     | 1.449     | 0.6931    |
hyperparameters : {'max_depth': 5, 'subsample': 0.6261564417044092, 'colsample_bytree': 0.8763145220620449, 'min_child_weight': 5.135323353557588, 'gamma': 8.39495450163982, 'reg_alpha': 8.950443047087845, 'reg_lambda': 1.4235649099168255, 'scale_pos_weight': 1.5217625173811569, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


c:\Users\Yuchie\Desktop\DataScience\kaggle_env\Lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


Gini Coefficient : 0.28457181017567373

| 8         | 0.2846    | 0.8763    | 8.395     | 4.561     | 5.135     | 8.95      | 1.424     | 1.522     | 0.6262    |
hyperparameters : {'max_depth': 6, 'subsample': 0.857971740304964, 'colsample_bytree': 0.9583821245229369, 'min_child_weight': 6.158305055403563, 'gamma': 9.305332775334449, 'reg_alpha': 8.200928434091152, 'reg_lambda': 1.2571039588093065, 'scale_pos_weight': 1.4700266933495618, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


c:\Users\Yuchie\Desktop\DataScience\kaggle_env\Lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


Gini Coefficient : 0.28538973194158385

| 9         | 0.2854    | 0.9584    | 9.305     | 5.594     | 6.158     | 8.201     | 1.257     | 1.47      | 0.858     |


In [16]:
max_params = optimizer.max['params']
max_params

{'colsample_bytree': 0.9583821245229369,
 'gamma': 9.305332775334449,
 'max_depth': 5.594282602920541,
 'min_child_weight': 6.158305055403563,
 'reg_alpha': 8.200928434091152,
 'reg_lambda': 1.2571039588093065,
 'scale_pos_weight': 1.4700266933495618,
 'subsample': 0.857971740304964}

In [17]:
max_params['max_depth'] = int(round(max_params['max_depth']))

max_params.update(fixed_params)
max_params

{'colsample_bytree': 0.9583821245229369,
 'gamma': 9.305332775334449,
 'max_depth': 6,
 'min_child_weight': 6.158305055403563,
 'reg_alpha': 8.200928434091152,
 'reg_lambda': 1.2571039588093065,
 'scale_pos_weight': 1.4700266933495618,
 'subsample': 0.857971740304964,
 'objective': 'binary:logistic',
 'learning_rate': 0.02,
 'random_state': 1991}

In [18]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

oof_val_preds = np.zeros(X.shape[0]) 
oof_test_preds = np.zeros(X_test.shape[0]) 

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print('#'*40, f'Fold {idx+1} / fold {folds.n_splits}', '#'*40)
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
    dtest = xgb.DMatrix(X_test)
    xgb_model = xgb.train(params=max_params, 
                          dtrain=dtrain,
                          num_boost_round=2000,
                          evals=[(dvalid, 'valid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=100)

    best_iter = xgb_model.best_iteration
    oof_test_preds += xgb_model.predict(dtest,
                                        iteration_range=(0, best_iter))/folds.n_splits
    
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid, 
                                                  iteration_range=(0, best_iter))
    
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'Fold {idx+1} gini coefficient : {gini_score}\n')

######################################## Fold 1 / fold 5 ########################################


c:\Users\Yuchie\Desktop\DataScience\kaggle_env\Lib\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	valid-logloss:0.67671	valid-gini:0.16215
[100]	valid-logloss:0.19182	valid-gini:0.24941
[200]	valid-logloss:0.15837	valid-gini:0.27811
[300]	valid-logloss:0.15506	valid-gini:0.28780
[400]	valid-logloss:0.15452	valid-gini:0.29319
[500]	valid-logloss:0.15439	valid-gini:0.29533
[600]	valid-logloss:0.15433	valid-gini:0.29681
[700]	valid-logloss:0.15427	valid-gini:0.29816
[800]	valid-logloss:0.15425	valid-gini:0.29872
[900]	valid-logloss:0.15422	valid-gini:0.29916
[1000]	valid-logloss:0.15422	valid-gini:0.29949
[1100]	valid-logloss:0.15422	valid-gini:0.29951
[1200]	valid-logloss:0.15422	valid-gini:0.29941
[1300]	valid-logloss:0.15420	valid-gini:0.29947
[1338]	valid-logloss:0.15421	valid-gini:0.29945
Fold 1 gini coefficient : 0.2996383794197772

######################################## Fold 2 / fold 5 ########################################
[0]	valid-logloss:0.67670	valid-gini:0.15338
[100]	valid-logloss:0.19190	valid-gini:0.24056
[200]	valid-logloss:0.15862	valid-gini:0.26445
[300]	val

In [19]:
print('OOF validation prediction gini coefficient:', eval_gini(y, oof_val_preds))

OOF validation prediction gini coefficient: 0.28905038063620453


In [20]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')